In [1]:
!pip install mediapipe opencv-python fastapi uvicorn python-multipart

   ---------------------------------------- 0.0/95.6 kB ? eta -:--:--
   ---------------------------------------- 95.6/95.6 kB 5.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/66.4 kB ? eta -:--:--
   ---------------------------------------- 66.4/66.4 kB ? eta 0:00:00
   ---------------------------------------- 0.0/72.7 kB ? eta -:--:--
   ---------------------------------------- 72.7/72.7 kB ? eta 0:00:00



[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# Instalar primero:
# !pip install mediapipe opencv-python fastapi uvicorn python-multipart

import mediapipe as mp
import cv2
import json
import math
from fastapi import FastAPI, File, UploadFile
import uvicorn
import numpy as np
from collections import defaultdict

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

# Función para calcular ángulo entre tres puntos
def calcular_angulo(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angulo = np.abs(radians * 180.0 / np.pi)
    if angulo > 180.0:
        angulo = 360 - angulo
    return angulo

# Obtener métricas usando MediaPipe Pose
def obtener_metricas(frame, frame_id):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resultados = pose.process(rgb_frame)
    metricas = {}

    if resultados.pose_landmarks:
        landmarks = resultados.pose_landmarks.landmark

        # Coordenadas necesarias
        cadera_d = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y]
        rodilla_d = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].y]
        tobillo_d = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].y]

        hombro_d = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y]
        hombro_i = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
        cadera_i = [landmarks[mp_pose.PoseLandmark.LEFT_HIP].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP].y]
        rodilla_i = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y]
        tobillo_i = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y]

        # Ángulos adicionales
        angulo_rodilla_derecha = calcular_angulo(cadera_d, rodilla_d, tobillo_d)
        angulo_rodilla_izquierda = calcular_angulo(cadera_i, rodilla_i, tobillo_i)
        angulo_espalda = calcular_angulo(hombro_d, cadera_d, rodilla_d)  # Inclinación torso
        angulo_cadera = calcular_angulo(hombro_d, cadera_d, tobillo_d)   # Alineación hombro-cadera-tobillo

        metricas = {
            "frame": frame_id,
            "angulo_rodilla_derecha": angulo_rodilla_derecha,
            "angulo_rodilla_izquierda": angulo_rodilla_izquierda,
            "angulo_espalda": angulo_espalda,
            "angulo_cadera": angulo_cadera
        }

    return metricas

# Detectar repeticiones simples por ángulo mínimo de rodilla (ejemplo rudimentario)
def detectar_repeticiones(data, umbral=90):
    repeticiones = []
    en_repeticion = False
    inicio = 0

    for i, frame in enumerate(data):
        angulo = min(frame['angulo_rodilla_derecha'], frame['angulo_rodilla_izquierda'])

        if angulo < umbral and not en_repeticion:
            inicio = frame['frame']
            en_repeticion = True
        elif angulo >= umbral and en_repeticion:
            fin = frame['frame']
            repeticiones.append((inicio, fin))
            en_repeticion = False

    return repeticiones

# Agrupar por fases y obtener resumen por fase
def agrupar_por_fases(data, repeticiones):
    resumen = {}
    for idx, (ini, fin) in enumerate(repeticiones):
        rango = fin - ini
        if rango <= 0:
            continue
        fases = {
            'fase1': [], 'fase2': [], 'fase3': [], 'fase4': []
        }
        for frame in data:
            if ini <= frame['frame'] <= fin:
                pos = frame['frame'] - ini
                if pos < 0.25 * rango:
                    fases['fase1'].append(frame)
                elif pos < 0.5 * rango:
                    fases['fase2'].append(frame)
                elif pos < 0.75 * rango:
                    fases['fase3'].append(frame)
                else:
                    fases['fase4'].append(frame)

        resumen[f'rep{idx+1}'] = {}
        for fase, frames in fases.items():
            if not frames:
                continue
            resumen[f'rep{idx+1}'][fase] = {
                'angulo_rodilla': {
                    'min': min(min(f['angulo_rodilla_derecha'], f['angulo_rodilla_izquierda']) for f in frames),
                    'max': max(max(f['angulo_rodilla_derecha'], f['angulo_rodilla_izquierda']) for f in frames)
                },
                'angulo_cadera': {
                    'min': min(f['angulo_cadera'] for f in frames),
                    'max': max(f['angulo_cadera'] for f in frames)
                },
                'angulo_espalda': {
                    'min': min(f['angulo_espalda'] for f in frames),
                    'max': max(f['angulo_espalda'] for f in frames)
                }
            }
    return resumen

# Opción 1: Cargar video local y obtener métricas
def analizar_video_local(ruta):
    video = cv2.VideoCapture(ruta)
    resultados = []
    frame_id = 0

    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break

        metricas_frame = obtener_metricas(frame, frame_id)
        if metricas_frame:
            resultados.append(metricas_frame)

        frame_id += 1

    video.release()
    return resultados

# Prueba local
video_path = "sentadilla.mp4"  # Coloca aquí tu video local
resultados = analizar_video_local(video_path)
reps = detectar_repeticiones(resultados)
resumen_reps = agrupar_por_fases(resultados, reps)

with open("resumen_por_reps.json", "w") as f:
    json.dump(resumen_reps, f, indent=2)

# Opción 2: Recibir video del frontend con FastAPI
app = FastAPI()

@app.post("/analizar_video/")
async def analizar_video(file: UploadFile = File(...)):
    contenido = await file.read()
    np_video = np.frombuffer(contenido, np.uint8)
    video = cv2.imdecode(np_video, cv2.IMREAD_COLOR)

    metricas = obtener_metricas(video, frame_id=0)
    return metricas

# Para ejecutar la API (descomenta la siguiente línea y ejecútala en consola)
# uvicorn.run(app, host="0.0.0.0", port=8000)
